## Reinforcement Learning
Reinforcement learning is a type of machine learning which involves an agent in an environment with a set of states $S$ , and a set of actions $A$. By performing an action $a \in A$, the environment transitions from state to state. Executing an action in a specific state provides the agent with a reward $r$.

The goal of the agent is to maximize its total (future) reward. It does this by adding the maximum reward attainable from future states to the reward for achieving its current state, effectively influencing the current action by the potential future reward. This potential reward is a weighted sum of the expected values of the rewards of all future steps starting from the current state.


## Q Learning
Q-learning is a model-free reinforcement learning algorithm (ie, no transition probability distribution and reward function). The goal of Q-learning is to learn a policy, which tells an agent what action to take under what circumstances. It does not require a model of the environment, and it can handle problems with stochastic transitions and rewards, without requiring adaptations.

For any finite Markov decision process (FMDP), Q-learning finds a policy that is optimal in the sense that it maximizes the expected value of the total reward over any and all successive steps, starting from the current state. Q-learning can identify an optimal action-selection policy for any given FMDP, given infinite exploration time and a partly-random policy. "Q" names the function that returns the reward used to provide the reinforcement and can be said to stand for the "quality" of an action taken in a given state.

**ALGORITHM:**  
The weight for a step from a state $\Delta t$ steps into the future is calculated as $\gamma^{\Delta t}$.  
where $\gamma$ (gamma - the discount factor) is a number between 0 and 1. It has the effect of valuing rewards received earlier higher than those received later.

The algorithm, therefore, has a function that calculates the quality of a state-action combination:  
    $$Q : S \ast A \to R$$

Before learning begins, $Q$ is initialized to a possibly arbitrary fixed value. Then, at each time $t$ the agent selects an action $a_t$, observes a reward $r_t$, enters a new state $s_{t+1}$ (that may depend on both the previous state $s_t$ and the selected action $a_t$), and $Q$ is updated. The core of the algorithm is a simple value iteration update, using the weighted average of the old value and the new information:

   $$Q_{new}(s_t, a_t) \gets (1-\alpha).Q(s_t, a_t) + \alpha.[r_t + \gamma. max_a Q(s_{t+1}, a)]$$
where $r_t$ is the reward received when moving from the state $s_t$ to the state $s_{t+1}$, and $\alpha$ is the learning rate $( 0 < \alpha \leq 1)$.

An episode of the algorithm ends when state $s_{t+1}$ is a final or terminal state. However, Q-learning can also learn in non-episodic tasks. If the discount factor is lower than 1, the action values are finite even if the problem can contain infinite loops.

For all final states $S_f$,  $Q(s_{f},a)$ is never updated, but is set to the reward value $r$ observed for state $S_f$. In most cases, $Q(s_f, a)$ can be taken to equal zero. 


**INFLUENCE OF VARIABLES:**
* Learning Rate $(\alpha)$: The learning rate or step size determines to what extent newly acquired information overrides old information.
* Explore vs Exploit: Helps in learning optimal policy by first exploring states then exploiting learned Q values.
* Discount factor: The discount factor $\gamma$ determines the importance of future rewards.


**IMPLEMENTATION:**  
Q-learning at its simplest stores data in tables. This approach falters with increasing numbers of states/actions. 

* Function approximation: 
Q-learning can be combined with function approximation. This makes it possible to apply the algorithm to larger problems, even when the state space is continuous.

One solution is to use an artificial neural network as a function approximator. Function approximation may speed up learning in finite problems, due to the fact that the algorithm can generalize earlier experiences to previously unseen states. 

* Quantization:
Another technique to decrease the state/action space quantizes possible values.


**VARIANTS:**
* Deep Q Learning:
In this method we will use Deep Neural Networks to estimate function approximation. Reinforcement learning is unstable or divergent when a nonlinear function approximator such as a neural network is used to represent Q. This instability comes from the correlations present in the sequence of observations, the fact that small updates to Q may significantly change the policy and the data distribution, and the correlations between Q and the target values.

In order to solve this instability we will use a technique called experience replay, a biologically inspired mechanism that uses a random sample of prior actions instead of the most recent action to proceed. This removes correlations in the observation sequence and smooths changes in the data distribution.

* Double Q-learning:
Because the future maximum approximated action value in Q-learning is evaluated using the same Q function as in current action selection policy, in noisy environments Q-learning can sometimes overestimate the action values, slowing the learning. A variant called Double Q-learning was proposed to correct this. Double Q-learning is an off-policy reinforcement learning algorithm, where a different policy is used for value evaluation than what is used to select the next action.

In practice, two separate value functions are trained in a mutually symmetric fashion using separate experiences, $Q_A$ and $Q_B$.

## Let's implement a DQN model to play CartPole game from OpenAI gym

* In this game we have 4 real values to represent a state and 2 possible actions (0 = left and 1 = right) in each state.
* reward: 1 for every step taken, including the termination step
* starting state: All observations are assigned a uniform random value between ±0.05
* episode termination: 1.Pole Angle is more than ±12°, 2.Cart Position is more than ±2.4 (center of the cart reaches the edge of the display), 3.Episode length is greater than 200
* success: considered solved when the average reward is greater than or equal to 195.0 over 100 consecutive trials.

In [1]:
# imports
import gym
import random
import math
import numpy as np
from collections import deque


# training parameters
num_episodes = 1000
num_win_ticks = 195
max_env_steps = None

# discount factor
gamma = 1.0
# exploration factor
epsilon = 1.0
epsilon_min = 0.01
epsilon_decay = 0.985
# learning rate
lr = 0.005

batch_size = 256
monitor = False
quiet = False

# environment parameters
memory = deque(maxlen=5000)
env = gym.make('CartPole-v0')
if max_env_steps is not None:
    env.max_episode_steps = max_env_steps


# Building the neural network
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# model
model = Sequential()
model.add(Dense(24, input_dim=4, activation='relu'))
model.add(Dense(24, activation='relu'))
model.add(Dense(2, activation='relu'))

model.compile(loss='mse', optimizer=Adam(lr=lr))


# define util funtions
def remember(state, action, reward, next_state, done):
    memory.append((state, action, reward, next_state, done))


def choose_action(state, epsilon):
    if(np.random.random() <= epsilon):
        return env.action_space.sample()
    else:
        return np.argmax(model.predict(state))


def get_epsilon(t):
    return max(epsilon_min, min(epsilon, 1.0 - math.log10(t+1) * epsilon_decay))


def preprocess_state(state):
    return np.reshape(state, [1,4])


def replay(batch_size, epsilon):
    x_batch = []
    y_batch = []
    mini_batch = random.sample(memory, min(len(memory), batch_size))
    
    for state, action, reward, next_state, done in mini_batch:
        y_target = model.predict(state)
        if(done):
            y_target[0][action] = reward
        else:
            y_target[0][action] = reward + gamma * np.max(model.predict(next_state)[0])
        x_batch.append(state[0])
        y_batch.append(y_target[0])
    
    model.fit(np.array(x_batch), np.array(y_batch), batch_size=1, verbose=0)
    
    if(epsilon > epsilon_min):
        epsilon *= epsilon_decay


# train function
def train():
    scores = deque(maxlen=100)
    
    for episode in range(num_episodes):
        state = preprocess_state(env.reset())
        done = False
        t = 0
        while(not done):
            action = choose_action(state, get_epsilon(episode))
            next_state, reward, done, info = env.step(action)
            next_state = preprocess_state(next_state)
            remember(state, action, reward, next_state, done)
            state = next_state
            t += 1
        scores.append(t)
        mean_score = np.mean(scores)
        
        if(mean_score>=num_win_ticks and episode>=100):
            if(not quiet):
                print('Ran {} episodes. Solved after {} trails'.format(episode, episode-100))
            env.close()
            return episode-100
        if(episode%20 == 0 and not quiet):
            print('Episode {}: mean score={}'.format(episode, mean_score))
        
        replay(batch_size, get_epsilon(episode))
    
    if(not quiet):
        print('Didn\'t solve after {} episodes.'.format(episode+1))
    env.close()
    return episode


train()

Using TensorFlow backend.


Episode 0: mean score=13.0
Episode 20: mean score=10.476190476190476
Episode 40: mean score=19.51219512195122
Episode 60: mean score=31.131147540983605
Episode 80: mean score=46.04938271604938
Episode 100: mean score=53.08
Episode 120: mean score=71.52
Episode 140: mean score=89.93
Episode 160: mean score=108.7
Episode 180: mean score=121.33
Episode 200: mean score=134.15
Episode 220: mean score=148.23
Episode 240: mean score=159.74
Episode 260: mean score=161.18
Episode 280: mean score=159.56
Episode 300: mean score=166.03
Episode 320: mean score=167.18
Episode 340: mean score=169.28
Episode 360: mean score=176.81
Episode 380: mean score=186.32
Episode 400: mean score=190.13
Episode 420: mean score=194.39
Ran 427 episodes. Solved after 327 trails


327

In [2]:
def play():
    for chance in range(5):
        state = preprocess_state(env.reset())
        done = False
        t = 0
        while(not done):
            action = np.argmax(model.predict(state))
            next_state, reward, done, info = env.step(action)
            env.render()
            next_state = preprocess_state(next_state)
            state = next_state
            t += 1
        print('Chance {}: score={}'.format(chance+1, t))

play()
env.close()

Chance 1: score=200
Chance 2: score=200
Chance 3: score=200
Chance 4: score=200
Chance 5: score=200


Great! For all the trials we got a score of 200 which is the maximum score for this game.